In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model

In [55]:
df_train = pd.read_csv("Train.csv")
df_test = pd.read_csv("Test.csv")
df_test['UnitPrice']=-1
df = pd.concat([df_train,df_test],axis=0)

C:\Users\ThisPc\anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [43]:
df.head()

,Country,CustomerID,Description,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,35,14056.0,144,2011-05-06 16:54:00,6141,3,1583,3.75
1,35,13098.0,3682,2011-05-11 07:35:00,6349,6,1300,1.95
2,35,15044.0,1939,2011-11-20 13:20:00,16783,4,2178,5.95
3,35,15525.0,2983,2011-11-22 12:07:00,16971,1,2115,0.83
4,35,13952.0,2886,2011-05-06 09:00:00,6080,12,1210,1.65


In [44]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 122048
Data columns (total 8 columns):
Country        406829 non-null int64
CustomerID     406829 non-null float64
Description    406829 non-null int64
InvoiceDate    406829 non-null object
InvoiceNo      406829 non-null int64
Quantity       406829 non-null int64
StockCode      406829 non-null int64
UnitPrice      406829 non-null float64
dtypes: float64(2), int64(5), object(1)
memory usage: 27.9+ MB
None


In [5]:
print(df.describe())

             Country     CustomerID    Description      InvoiceNo  \
count  406829.000000  406829.000000  406829.000000  406829.000000   
mean       32.827731   15287.690570    2024.692726    9950.901755   
std         6.738565    1713.600303    1089.177346    5552.295845   
min         0.000000   12346.000000       0.000000       0.000000   
25%        35.000000   13953.000000    1143.000000    5060.000000   
50%        35.000000   15152.000000    1987.000000   10310.000000   
75%        35.000000   16791.000000    2945.000000   14651.000000   
max        36.000000   18287.000000    3895.000000   22189.000000   

            Quantity      StockCode      UnitPrice  
count  406829.000000  406829.000000  406829.000000  
mean       12.061303    1573.257501       2.115848  
std       248.693370     844.054584      65.625402  
min    -80995.000000       0.000000      -1.000000  
25%         2.000000     937.000000      -1.000000  
50%         5.000000    1522.000000       1.250000  
75%    

In [56]:
df.head()

,Country,CustomerID,Description,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,35,14056.0,144,2011-05-06 16:54:00,6141,3,1583,3.75
1,35,13098.0,3682,2011-05-11 07:35:00,6349,6,1300,1.95
2,35,15044.0,1939,2011-11-20 13:20:00,16783,4,2178,5.95
3,35,15525.0,2983,2011-11-22 12:07:00,16971,1,2115,0.83
4,35,13952.0,2886,2011-05-06 09:00:00,6080,12,1210,1.65


In [57]:
#dropping customerID and invoiceNo because as such they are not contributing to target varivable
df = df.drop(["CustomerID","InvoiceNo","InvoiceDate","Country"],axis = 1)

In [58]:
df.head()

,Description,Quantity,StockCode,UnitPrice
0,144,3,1583,3.75
1,3682,6,1300,1.95
2,1939,4,2178,5.95
3,2983,1,2115,0.83
4,2886,12,1210,1.65


In [59]:
df.isnull().sum()

Description    0
Quantity       0
StockCode      0
UnitPrice      0
dtype: int64

In [60]:
df['Quantity'] = df['Quantity'].apply(lambda x: abs(x))

In [9]:
print(df.describe())

             Country    Description       Quantity      StockCode  \
count  406829.000000  406829.000000  406829.000000  406829.000000   
mean       32.827731    2024.692726      13.412279    1573.257501   
std         6.738565    1089.177346     248.624170     844.054584   
min         0.000000       0.000000       1.000000       0.000000   
25%        35.000000    1143.000000       2.000000     937.000000   
50%        35.000000    1987.000000       5.000000    1522.000000   
75%        35.000000    2945.000000      12.000000    2106.000000   
max        36.000000    3895.000000   80995.000000    3683.000000   

           UnitPrice  
count  406829.000000  
mean        2.115848  
std        65.625402  
min        -1.000000  
25%        -1.000000  
50%         1.250000  
75%         2.550000  
max     38970.000000  


In [61]:
df.corr()

,Description,Quantity,StockCode,UnitPrice
Description,1.000000,0.000728,-0.011504,-0.000387
Quantity,0.000728,1.000000,0.001495,-0.000959
StockCode,-0.011504,0.001495,1.000000,0.016785
UnitPrice,-0.000387,-0.000959,0.016785,1.000000


In [ ]:
df['Country'].value_counts()

In [ ]:
p = sns.pairplot(df)

In [11]:
date_column = pd.to_datetime(df['InvoiceDate'])

In [12]:
print(date_column)

0        2011-05-06 16:54:00
1        2011-05-11 07:35:00
2        2011-11-20 13:20:00
3        2011-11-22 12:07:00
4        2011-05-06 09:00:00
                 ...        
122044   2011-06-09 09:44:00
122045   2011-06-28 15:47:00
122046   2010-12-13 12:26:00
122047   2011-06-16 12:45:00
122048   2011-09-12 13:50:00
Name: InvoiceDate, Length: 406829, dtype: datetime64[ns]


In [13]:
#creating new dataframe 
dateTimeStamp_df = pd.DataFrame({"year": date_column.dt.year,
              "month": date_column.dt.month,
              "day": date_column.dt.day,
              #"hour": date_column.dt.hour,
              #"min": date_column.dt.minute,
              #"second": date_column.dt.second,
              #"weekday": date_column.dt.weekday,
             })

In [14]:
dateTimeStamp_df.head()

,year,month,day
0,2011,5,6
1,2011,5,11
2,2011,11,20
3,2011,11,22
4,2011,5,6


In [15]:
df = pd.concat([df,dateTimeStamp_df],axis=1)

In [47]:
df.head()

,Country,Description,InvoiceDate,Quantity,StockCode,UnitPrice
0,35,144,2011-05-06 16:54:00,3,1583,3.75
1,35,3682,2011-05-11 07:35:00,6,1300,1.95
2,35,1939,2011-11-20 13:20:00,4,2178,5.95
3,35,2983,2011-11-22 12:07:00,1,2115,0.83
4,35,2886,2011-05-06 09:00:00,12,1210,1.65


In [48]:
#dropping InvoiceDate because we have taken information and keep in a separte dataframe
df_new = df.drop(["InvoiceDate"],axis = 1)

In [62]:
#df_new_only_train = df_new[df_new['UnitPrice'] != -1]
df_new_only_train = df[df['UnitPrice'] != -1]

In [63]:
df_new_only_train.tail()

,Description,Quantity,StockCode,UnitPrice
284775,1081,6,1706,1.65
284776,2947,16,2184,0.65
284777,3632,4,667,8.50
284778,3089,48,1576,3.45
284779,1156,2,2603,0.75


In [72]:
y = df_new_only_train['UnitPrice']
X = df_new_only_train.drop(['UnitPrice'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)
## Train Test Split

In [73]:
#Regression on everything
from sklearn.ensemble import RandomForestRegressor

cls = RandomForestRegressor(n_estimators=50,n_jobs=2, random_state=42)
cls.fit(X_train, y_train)
pred = cls.predict(X_test)

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_test, pred))
print(rmse_train)

136.8509668094604


In [75]:
df_new_only_test = df[df['UnitPrice'] == -1]
df_new_test = df_new_only_test.drop(["UnitPrice"],axis = 1)
df_new_test.head()

,Description,Quantity,StockCode
0,1079,3,1709
1,3457,1,510
2,694,36,604
3,3473,2,1478
4,871,1,3216


In [76]:
#Regression on everything
from sklearn.ensemble import RandomForestRegressor

cls = RandomForestRegressor(n_estimators=50,n_jobs=2, random_state=42)
cls.fit(X, y)
pred = cls.predict(df_new_test)
# predict
pd.DataFrame(pred).to_csv("submission1.csv")

In [ ]:
#3 Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()

X = sc_X.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

In [ ]:
#4 Fitting the Support Vector Regression Model to the dataset
# Create your support vector regressor here
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train)

In [ ]:
from sklearn import svm

svc = svm.SVC(kernel='linear', C=1,gamma=1).fit(X_train, y_train)

Z = svc.predict(X_test)

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_test, Z))
print(rmse_train)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=3)
  
  # transforms the existing features to higher degree features.
X_train_poly = poly_features.fit_transform(X_train)
  
  # fit the transformed features to Linear Regression
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
  
  # predicting on training data-set
y_train_predicted = poly_model.predict(X_train_poly)
  
  # predicting on test data-set
y_test_predict = poly_model.predict(poly_features.fit_transform(X_test))
  
  # evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_test, y_test_predict))
print(rmse_train)

In [ ]:
import statsmodels.api as sm

# with statsmodels
X_ols = sm.add_constant(X_train) # adding a constant
 
model = sm.OLS(y_train, X_ols).fit()
predictions = model.predict(sm.add_constant(X_test)) 

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_test, predictions))
print(rmse_train)

print_model = model.summary()
print(print_model)

In [ ]:
# with statsmodels
trainX = X_train.drop(['day','Description','hour','min','second','Quantity'],axis = 1)
testX = X_test.drop(['day','Description','hour','min','second','Quantity'],axis = 1)
X_ols = sm.add_constant(trainX) # adding a constant
 
model = sm.OLS(y_train, X_ols).fit()
predictions = model.predict(sm.add_constant(testX)) 

# evaluating the model on training dataset
rmse_train = np.sqrt(mean_squared_error(y_test, predictions))
print(rmse_train)

print_model = model.summary()
print(print_model)

In [78]:
def normalization(X_train, X_test):
    from sklearn import preprocessing
    scaler = preprocessing.StandardScaler().fit(X_train) 
    X_train_std = scaler.transform(X_train)
    X_test_std = scaler.transform(X_test)
    return X_train_std, X_test_std, scaler

In [79]:
#Lasso regression is L1 regression and is often used for feature selection as well
#Note that sklearn offers built-in cross val
def lassoCV_reg_workflow(X,y):
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.linear_model import LassoCV
    #split into test/train and normalize
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state=42)
   # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, random_state=10)
    X_train, X_test, scaler = normalization(X_train,X_test)
    #sklearn has trouble with panda df, convert to matrix
    y_train,y_test = y_train.values.ravel(),y_test.values.ravel()
    #set of alphas we use to cross validate
    alphas = np.logspace(-4, 4, 14) #10**start, 10**end, num_samples,
    lasso_cv = LassoCV(max_iter=10**6,alphas=alphas)
    lasso_cv.fit(X_train,y_train)
    y_pred = lasso_cv.predict(X_test)
    
    print("Lasso CV Mean squared error:", mean_squared_error(y_test, y_pred))
    print('Lasso CV Variance score aka r^2:', r2_score(y_test, y_pred))
    print('Lasso CV Root Mean squared error:',np.sqrt(mean_squared_error(y_test, y_pred)))
    coefficients = lasso_cv.coef_
    #best alpha chosen by cv
    alpha = lasso_cv.alpha_ 
    return y_test,y_pred,coefficients,alpha

lasso_y_test,lasso_y_pred,lasso_coef,lasso_alpha = lassoCV_reg_workflow(X,y)
print('Lasso CV Regression coefficients', lasso_coef)
print('Lasso CV Regression optimal alpha', lasso_alpha)

Lasso CV Mean squared error: 56135.44674708455
Lasso CV Variance score aka r^2: 0.00014552128239242457
Lasso CV Root Mean squared error: 236.9292019719911
Lasso CV Regression coefficients [-0.03948532 -0.07410937  1.1267137 ]
Lasso CV Regression optimal alpha 0.00701703828670383


In [80]:
#Ridge regression is L2 regression and tends to offer higher accuracy than L1
def ridgeCV_reg_workflow(X,y):
    from sklearn.metrics import mean_squared_error, r2_score
    from sklearn.linear_model import RidgeCV
    #split into test/train and normalize
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state=42)
    X_train, X_test, scaler = normalization(X_train,X_test)
    #another method to convert panda dataframes into normal arrays
    y_train,y_test = y_train.values.ravel(),y_test.values.ravel()
    alphas = np.logspace(-4, 4, 14) #10**start, 10**end,num_samples,
    
    ridge_cv = RidgeCV(alphas=alphas)
    ridge_cv.fit(X_train,y_train)
    y_pred = ridge_cv.predict(X_test)
    
    print("Ridge CV Mean squared error:", mean_squared_error(y_test, y_pred))
    print('Ridge CV Variance score aka r^2:', r2_score(y_test, y_pred))
    print('Ridge CV Root Mean squared error:',np.sqrt(mean_squared_error(y_test, y_pred)))
    coef = ridge_cv.coef_
    alpha = ridge_cv.alpha_
    return y_test,y_pred,coef,alpha,ridge_cv

ridge_y_test,ridge_y_pred,ridge_coef,ridge_alpha,ridge_cv = ridgeCV_reg_workflow(X,y)
print('Ridge CV Regression coefficients', ridge_coef)
print('Ridge CV Regression optimal alpha', ridge_alpha)

Ridge CV Mean squared error: 56135.483890396405
Ridge CV Variance score aka r^2: 0.000144859705637157
Ridge CV Root Mean squared error: 236.92928035681112
Ridge CV Regression coefficients [-0.04611022 -0.08035653  1.12303115]
Ridge CV Regression optimal alpha 2424.462017082331


In [ ]:

## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [ ]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

In [ ]:
classifier=xgboost.XGBClassifier()

In [ ]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=2,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
random_search.fit(X_train,y_train)
# X_train, X_test, y_train, y_test

In [ ]:
random_search.best_estimator_